In [1]:
import re
from itertools import chain
from tqdm import tqdm
import numpy as np

DICT = []

STATE_WORD = 0
STATE_TYPE = 1
STATE_MEANING_START = 2

class DictProcessor():
    def __init__(self):
        self.word = ""
        self.meaning = ""
        
        self.state = STATE_WORD
        
        self.dict = []

    def update(self, line):
        if line == "":
            self.go(STATE_WORD)
            return    
        elif re.match(r"^\s*\d+\.(.*)", line):
            m = re.match(r"^\s*\d+\.(.*)", line)
            line = m.group(1)
            self.go(STATE_TYPE)
        elif re.match(r"^\s*\d+\)", line):   
            m = re.match(r"^\s*\d+\)(.*)", line)
            line = m.group(1)
            self.go(STATE_MEANING_START)
        
        self._process(line)
        
    def _process(self, line):
        if self.state == STATE_WORD:
            self.word = line
            self.go(STATE_TYPE)
        elif self.state == STATE_TYPE:
            pass
            self.go(STATE_MEANING_START)
        elif self.state == STATE_MEANING_START:
            self.meaning += " " + line.strip()
    
    def _publish(self):
        self.meaning = self.meaning.strip()
        if self.word != "" and self.meaning != "" and len(self.word) >= 3:
            self.dict.append((self.word, self.meaning))
            
            #print("!!!| [%s] - [%s]" % (self.word, self.meaning))        
        self.meaning = ""                        
    
    def go(self, new_state):
        if self.state == STATE_MEANING_START:
            self._publish()
        
        self.state = new_state        

processor = DictProcessor()        
        
with open("efremova.txt") as fin:
    row = -1
    
    state = STATE_WORD
    
    word = ""
    for line in fin:
        row += 1
        
        if row < 3:
            continue
            
        if row > 100 and False:
            break
        
        line = line.strip()                
        
        #print("%d: %s" % (processor.state, line))
        
        processor.update(line)                        

In [2]:
print(len(processor.dict))

200438


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
words_tr = TfidfVectorizer(analyzer='char', ngram_range=(2,4))
f_words = words_tr.fit_transform((w_m[0] for w_m in processor.dict))

In [5]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
cache = {}

def preprocess(word):
    if word not in cache:
        cache[word] = morph.parse(word)[0].normal_form
        
    return cache[word]    

def tokenizer(line):
    res = []
    for w in re.findall("\w+", line):
        res.append(preprocess(w))
        
    return res    

meaning_tr = TfidfVectorizer(tokenizer=tokenizer)
f_meaning = meaning_tr.fit_transform((w_m[1] for w_m in processor.dict))

In [6]:
for i, (k, v) in enumerate(cache.items()):
    print("%s-%s" % (k, v))
    if i > 20:
        break

а-а
также-также
я-я
ть-ть
словообразовательная-словообразовательный
единица-единица
образующая-образующая
как-как
переходные-переходный
так-так
и-и
непереходные-непереходный
глаголы-глагол
несовершенного-несовершенный
вида-вид
с-с
общим-общий
значением-значение
действия-действие
которое-который
имеет-иметь
отношение-отношение


In [7]:
for i, w in enumerate(meaning_tr.get_feature_names()):
    if i > 100:
        print(w)
    if i > 300:
        break

19
1900
1903
1904
1905
1906
1907
1908
1910
1912
1914
1917
1918
1919
1920
1921
1922
1923
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1960
1966
1968
1971
1972
1979
1980
1989
1991
1993
1а1
1а2
1а3
1а4
1а5
1а6
1а7
1а8
2
20
200
206
21
210
212
22
23
239
24
2400
244
25
2500
26
27
28
29
2а1
2а2
2а3
2а4
2а5
3
30
300
3000
3048
32
330
335
34
35
356
36
360
3600
365
366
3а1
3а2
3а3
4
40
400
4047
409
44
45
450
453
46
480
489
491
4а2
5
50
500
52
54
58
5а1
5а2
6
60
600
6000
64
66
7
70
700
711
72
750
772
775
79
8
80
800
81
83
850
86
87
88
9
90
900
91
96
962
b
c
color
font
i
ii
iii
iv
ix
j
p
pluralia
r
red
s
tantum
v
vi
vii
viii
x
x1x
xi
xii
xiii
xiv
xix
xv
xvi
xvii
xviii
xx
а
абажур
абазин
абазинец
абазинский
абак
аббат
аббатиса
аббатисса
аббатство
аббревиатура
абвер
аберрация
абзац
абиогенез
абиогенный
абиссаль


In [8]:
print(f_meaning.shape)

(200438, 61895)


In [9]:
print(f_meaning[2, :])

  (0, 50215)	0.139517528718
  (0, 11974)	0.126804085358
  (0, 29542)	0.148867033309
  (0, 17059)	0.104321051909
  (0, 53371)	0.160321103939
  (0, 15264)	0.0721948287248
  (0, 27242)	0.218367549496
  (0, 8437)	0.134149793405
  (0, 27752)	0.203546461353
  (0, 4678)	0.114313103308
  (0, 47718)	0.065060408253
  (0, 15102)	0.127403068793
  (0, 54392)	0.198647783618
  (0, 59716)	0.133466553303
  (0, 24702)	0.126598343328
  (0, 23722)	0.283250347002
  (0, 8)	0.070986946694
  (0, 158)	0.081976616484
  (0, 3739)	0.0627078379595
  (0, 15948)	0.164774034023
  (0, 54081)	0.167292231629
  (0, 4329)	0.15990045476
  (0, 48739)	0.114948185391
  (0, 16645)	0.301636281446
  (0, 12228)	0.0864134946292
  (0, 20100)	0.088123690022
  (0, 16045)	0.265636023274
  (0, 53067)	0.286821266467
  (0, 41202)	0.162091685472
  (0, 3265)	0.221842579687
  (0, 59764)	0.244111269669
  (0, 13293)	0.211172995704
  (0, 28487)	0.24081517199


In [10]:
BAD_W = 1

from scipy.sparse import vstack, hstack
from random import randint

X = []
Y = []

for i in tqdm(range(f_words.shape[0] // 1)):
    f_w = f_words[i:i+1, :]
    f_m = f_meaning[i:i+1, :]
        
    Y.append(1)
    X.append(hstack([f_w, f_m]))
    for j in range(BAD_W):        
        rand_i = randint(0, f_words.shape[0] - 1)
        if rand_i == i:
            continue
        Y.append(0)    
        X.append(hstack([f_w, f_meaning[rand_i:rand_i+1, :]]))
        
    #break    
X = vstack(X)    
Y = np.array(Y)

100%|██████████| 200438/200438 [02:26<00:00, 1367.82it/s]


In [11]:
print(X.shape)
print(Y.shape)
print(f_words.shape)
print(f_meaning.shape)

(400876, 127518)
(400876,)
(200438, 65623)
(200438, 61895)


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [13]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [14]:
t = "svm"

if t == "svm":
    svm = LinearSVC()
    clf = CalibratedClassifierCV(svm) 
    
clf.fit(X_train, y_train)

y_proba = clf.predict_proba(X_test)

In [15]:
from sklearn.metrics import roc_auc_score
print(y_proba)
print(y_test)
print(roc_auc_score(y_test, y_proba[:, 1]))

[[ 0.57176645  0.42823355]
 [ 0.50361462  0.49638538]
 [ 0.4780626   0.5219374 ]
 ..., 
 [ 0.52640057  0.47359943]
 [ 0.51461076  0.48538924]
 [ 0.49991339  0.50008661]]
[1 0 1 ..., 1 0 0]
0.734991726862


In [18]:
#0.9 0.2 0.4 0.5
#1 1 0 0

diff = np.abs(y_proba[:, 1] - y_test)
diff_order = np.argsort(diff)[::-1] #diff[diff_order] - decreasing

N = 10

#print(diff[diff_order])
print(diff[diff_order[:N]])
print(y_proba[:, 1][diff_order[:N]])
print(y_test[diff_order[:N]])

docs_nums = diff_order[:N] // (1 + BAD_W)
print(docs_nums)

for i in docs_nums:
    print(processor.dict[i])

[ 0.79975882  0.79924419  0.79792354  0.79371854  0.79101221  0.78797627
  0.78739985  0.78705949  0.78550101  0.78548349]
[ 0.20024118  0.79924419  0.20207646  0.20628146  0.20898779  0.21202373
  0.21260015  0.21294051  0.78550101  0.78548349]
[1 0 1 1 1 1 1 1 0 0]
[13466  9853  9357  4438 11928  2836 10574 12862 10954   750]
('ветровой', 'Возникающий в результате действия ветра.')
('брать', 'разг. неперех. Захватывать приманку на рыболовном крючке, ловиться на удочку; клевать (о рыбе).')
('боров', 'перен. разг.-сниж. Толстый, неповоротливый человек.')
('бактерии', 'Одноклеточные микроорганизмы.')
('василёк', 'Травянистое растение семейства сложноцветных с голубыми или синими цветками, растущее обычно во ржи и в посевах других злаков.')
('апостольский', 'Соотносящийся по знач. с сущ.: апостол (2*), связанный с ним.')
('буерный', 'Соотносящийся по знач. с сущ.: буер, связанный с ним.')
('венчик', 'Часть цветка, состоящая из отдельных или сросшихся лепестков.')
('бург', 'Укрепленный пу